#### Summary
- torch.cuda.amp怎么使用
    - fp16: loss scaling
        - https://github.com/mli/transformers-benchmarks/blob/main/transformers.ipynb
    - 极大地提升batch_size

#### cnn pipeline

In [1]:
import torch
import torch.nn.functional as F
import torchvision.datasets as datasets
import torchvision.transforms as transforms

from torch import optim
from torch import nn
from torch.utils.data import DataLoader
from tqdm import tqdm

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

#### Model

In [3]:
# Simple CNN
class CNN(nn.Module):
    def __init__(self, in_channels=1, num_classes=10):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels=in_channels,
            out_channels=5120,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        # /2, downsampling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(
            in_channels=5120,
            out_channels=10240,
            kernel_size=3,
            stride=1,
            padding=1,
        )
        # (channels*w*h)
        # w, h: 取决于初始的 width, height
        self.fc1 = nn.Linear(10240 * 7 * 7, num_classes)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        # /2
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        # /2
        x = self.pool(x)
        print(x.shape)
        # 4d => 2d, (bs, features)
        x = x.reshape(x.shape[0], -1)
        x = self.fc1(x)
        return x

In [29]:
from torchsummary import summary

In [19]:
model = CNN(in_channels=3)
summary(model, input_size=(3, 224, 224), batch_size=32, device='cpu')

torch.Size([2, 5120, 224, 224])
torch.Size([2, 5120, 112, 112])
torch.Size([2, 10240, 56, 56])
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1       [32, 5120, 224, 224]         143,360
         MaxPool2d-2       [32, 5120, 112, 112]               0
            Conv2d-3      [32, 10240, 112, 112]     471,869,440
         MaxPool2d-4        [32, 10240, 56, 56]               0
            Linear-5                   [32, 10]     321,126,410
Total params: 793,139,210
Trainable params: 793,139,210
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 18.38
Forward/backward pass size (MB): 117600.00
Params size (MB): 3025.59
Estimated Total Size (MB): 120643.96
----------------------------------------------------------------


#### training pipeline

In [4]:
in_channels = 1
num_classes = 10

learning_rate = 3e-4
batch_size = 32
num_epochs = 3

In [21]:
train_dataset = datasets.MNIST(root='dataset/', train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root='dataset/', train=False, transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

100%|██████████| 9912422/9912422 [00:02<00:00, 4210621.16it/s] 


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 1010727.52it/s]


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw



100%|██████████| 1648877/1648877 [00:01<00:00, 1255326.08it/s]


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw



100%|██████████| 4542/4542 [00:00<00:00, 10986464.11it/s]

Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw



In [23]:
batch_x, batch_y = next(iter(train_loader))
print(batch_x.shape, batch_y.shape)

torch.Size([32, 1, 28, 28]) torch.Size([32])


#### float32

In [ ]:
in_channels = 1
num_classes = 10

learning_rate = 3e-4
batch_size = 128
num_epochs = 3

train_dataset = datasets.MNIST(root="dataset/", train=True, transform=transforms.ToTensor(), download=True)
test_dataset = datasets.MNIST(root="dataset/", train=False, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

model = CNN(in_channels=in_channels, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


def train():
    for epoch in tqdm(range(num_epochs)):
        for batch_idx, (batch_x, batch_y) in tqdm(enumerate(train_loader)):
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            logits = model(batch_x)
            loss = criterion(logits, batch_y)

            optimizer.zero_grad()
            loss.backward()

            optimizer.step()


def evalute(model, test_loader):
    total_correct = 0
    total_samples = 0
    model.eval()

    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            logits = model(batch_x)
            _, preds = logits.max(1)
            total_correct += (preds == batch_y).sum()
            total_samples += batch_y.size(0)
    model.train()
    return total_correct / total_samples


train()

  0%|          | 0/3 [00:00<?, ?it/s]
0it [00:00, ?it/s]

torch.Size([128, 10240, 7, 7])



1it [00:03,  3.41s/it]
2it [00:04,  1.85s/it]

torch.Size([128, 10240, 7, 7])



3it [00:06,  2.10s/it]

...

torch.Size([128, 10240, 7, 7])



144it [05:48,  2.42s/it]

torch.Size([128, 10240, 7, 7])



145it [05:50,  2.42s/it]

torch.Size([128, 10240, 7, 7])


In [ ]:
print(f"Accuracy on training set: {evalute(model, train_loader) * 100:.2f}")
print(f"Accuracy on test set: {evalute(model, test_loader) * 100:.2f}")

#### 混合精度训练

In [ ]:
in_channels = 1
num_classes = 10

learning_rate = 3e-4
batch_size = 256
num_epochs = 3

train_dataset = datasets.MNIST(
    root="dataset/", train=True, transform=transforms.ToTensor(), download=True
)
test_dataset = datasets.MNIST(
    root="dataset/", train=False, transform=transforms.ToTensor(), download=True
)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

model = CNN(in_channels=in_channels, num_classes=num_classes).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

scaler = torch.cuda.amp.GradScaler()


def train():
    for epoch in tqdm(range(num_epochs)):
        for batch_idx, (batch_x, batch_y) in tqdm(enumerate(train_loader)):
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            with torch.cuda.amp.autocast():
                logits = model(batch_x)
                loss = criterion(logits, batch_y)

            optimizer.zero_grad()
            scaler.scale(loss).backward()

            scaler.step(optimizer)
            scaler.update()


def evalute(model, test_loader):
    total_correct = 0
    total_samples = 0
    model.eval()

    with torch.no_grad():
        for batch_x, batch_y in test_loader:
            batch_x = batch_x.to(device)
            batch_y = batch_y.to(device)

            logits = model(batch_x)
            _, preds = logits.max(1)
            total_correct += (preds == batch_y).sum()
            total_samples += batch_y.size(0)
    model.train()
    return total_correct / total_samples


train()
